In [1]:
using HomotopyContinuation, HomologyInferenceWithWeakFeatureSize, DynamicPolynomials
hwfs = HomologyInferenceWithWeakFeatureSize
data_path = escape_string("paper_data/")
threshold = 1e-7
conversion_function = hwfs.reach_bertini_convert_solution_point_to_standard_form

# Order of bottlenecks, change for other orders
k = 2

filename_root = ""

# Uncomment if you have re-run the corresponding Bertini script
# data_path = escape_string("paper_data/bertini_inputs/")
# filename_root = ""

""

In [2]:
num_vars = 2
@polyvar q[1:num_vars]
x = q[1]
y = q[2]
F = [x^4 - x^2*y^2 + y^4 - 4*x^2 - 2*y^2 - x - 4*y + 1]
singular_points = hwfs.parse_bertini_file(data_path*filename_root*"singular_solutions")
singular_points = [conversion_function(point,num_vars,k) for point in singular_points]
nonsingular_points = hwfs.parse_bertini_file(data_path*filename_root*"nonsingular_solutions")
nonsingular_points = [conversion_function(point,num_vars,k) for point in nonsingular_points]

distance_system_for_filtering,parameters_for_filtering = hwfs.minimum_distance_start_system(F,q)
start = randn(ComplexF64,num_vars+length(F))
initial_system = subs(distance_system_for_filtering,parameters_for_filtering => start)
initial_solution = solutions(solve(initial_system,start_system = :polyhedral,parameters=:conservative))

function check_if_solution_has_real_circumcenter(point)
    for coordinate in point["circumcenter"]
        if abs(imag(coordinate)) >= threshold
            return false
        end
    end
    return true
end

filtered_nonsingular_points = [point for point in nonsingular_points if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)]

nondegenerate_points = [singular_points;filtered_nonsingular_points]
nondegenerate_points = [point for point in nondegenerate_points if check_if_solution_has_real_circumcenter(point)]
real_positive_D_value_points = [point for point in nondegenerate_points if hwfs.real_and_positive_d_value_check(point;threshold=threshold)]
can_be_distance_filtered = [point for point in real_positive_D_value_points if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)]
cannot_be_distance_filtered = [point for point in real_positive_D_value_points if !hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)]

distances_isolated = [Inf]
if length(can_be_distance_filtered) > 0
    distances_isolated = [hwfs.filter_solution_to_bottleneck(solution,length(q),k,distance_system_for_filtering,start,parameters_for_filtering,initial_solution;threshold=threshold,reach=true) for solution in can_be_distance_filtered]
    distances_isolated = [dist for dist in distances_isolated if dist!=false]
    if length(distances_isolated) == 0
        distances_isolated = [Inf]
    end
end
distances_not_isolated = [Inf]
if length(cannot_be_distance_filtered) > 0 
    distances_not_isolated = [real(sqrt(hwfs.apply_distance_squared_to_solution(point))) for point in cannot_be_distance_filtered]
end




In [3]:
# Critical values from non-isolated contributors
minimum(distances_isolated)

0.10362272960376587

In [4]:
# Critical values from isolated geometric k-bottlenecks
minimum(distances_not_isolated)

Inf